# MESSAGE-ix matrix improvement tool

This jupyter notebook is a prototype of the MESSAGE-ix matrix improvement tool.
This tool is aimed to automatically improve and optimize coefficient matrix quality for a MESSAGE-ix scenario and the used to return the results from scaled MESSAGE-ix matrix to the originally intended values.

This tool is derived from the tool developed by Makowski & Sosnowski, 1981 (https://pure.iiasa.ac.at/id/eprint/1766/1/CP-81-037.pdf)

According to Curtis and Reid (1972), matrix A can be described as well-scaled if:

$
\sum_{i} \sum_{j} \ (log_{10} \ |a_{i,j}|)^2 \leq v \qquad \qquad Eq. 1
$

with $v$ is an acceptable matrix quality criteria.
If $ax_{i,j}$ is $(log_{10} |a_{i,j}|)^2$ where $a_{i,j}$ is a non-zero value, matrix $Ax$ is:

In [1]:
import numpy as np
import pandas as pd
from pyomo.environ import *
from datetime import datetime
import xarray as xr

import matplotlib.pyplot as plt

def showme(df):
    return df["val"].unstack()

In [2]:
# Load the whole matrix
matrix       = (pd.read_csv('data/matrix_transport.csv')
               .set_index(['row','col'],drop=True)[['val']])
# Load and create matrix with small and large coefficient
matrix_up    = (pd.read_csv('data/matrix_transport_prob_upper.csv')
               .set_index(['row','col'],drop=True)[['val']])
matrix_lo    = (pd.read_csv('data/matrix_transport_prob_lower.csv')
               .set_index(['row','col'],drop=True)[['val']])
matrix_solv  = pd.concat([matrix_lo,matrix_up])


C:\Users\pratama\AppData\Local\Temp\ipykernel_21808\176877734.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matrix_solv  = pd.concat([matrix_lo,matrix_up])


In [3]:
matrix_solv

val
row  col                                
_obj x(san-diego,new-york)  2.500000e-09
     x(san-diego,topeka)    1.400000e-08

In [4]:
# calculate log base 10 of the absolute value of the matrix
log_absmatrix = matrix.copy()
log_absmatrix.loc[log_absmatrix['val']!=0,'val']=(np.log10(
    np.absolute(
        log_absmatrix.loc[log_absmatrix['val']!=0,'val'])))
#log_absmatrix = log_absmatrix.astype(int)

# calculate log base 10 of the absolute value of the matrix
log_absmatrix_solv = matrix_solv.copy()
log_absmatrix_solv.loc[log_absmatrix_solv['val']!=0,'val']=(np.log10(
    np.absolute(
        log_absmatrix_solv.loc[log_absmatrix_solv['val']!=0,'val'])))
#log_absmatrix_solv = log_absmatrix_solv.astype(int)

## Start Looping

In [5]:
# SCALE BY ROW
# Populating row scaler
RSFs = {row:[] for row in set(log_absmatrix_solv.index.get_level_values(0))}
for k in RSFs.keys():
    rval = log_absmatrix.loc[(k),"val"]
    lb,ub = min(rval),max(rval)
    mid = np.mean([lb,ub])
    RSFs[k] = 10**(-mid)

# Create DataFrame of row scaler
row_scaler = pd.DataFrame(data=RSFs, index=["val"]).transpose()
row_scaler.index.name = 'row'

# Create new matrix with scaled rows
matrix0 = matrix.copy()
matrix0.loc[row_scaler.index] = matrix0.loc[row_scaler.index].mul(row_scaler)

In [6]:
# SCALE BY COL
# Populating col scaler
log_absmatrix0 = np.log10(matrix0)
log_absmatrix0_solv = log_absmatrix0.loc[(log_absmatrix0["val"] >= 2) | 
                                         (log_absmatrix0["val"] <= -2)]

CSFs = {col:[] for col in set(log_absmatrix0_solv.index.get_level_values(1))}
for k in CSFs.keys():
    cval = log_absmatrix0.loc[(log_absmatrix0.index.get_level_values('col') == k),"val"]
    lb,ub = min(cval),max(cval)
    mid = np.mean([lb,ub])
    CSFs[k] = 10**(-mid)

# Create DataFrame of col scaler
col_scaler = pd.DataFrame(data=CSFs, index=["val"]).transpose()
col_scaler.index.name = 'col'

# Create new matrix with scaled rows
new_matrix = matrix0.copy()
new_matrix.loc[(new_matrix.index.get_level_values('col').isin(list(col_scaler.index)))] = new_matrix.loc[(new_matrix.index.get_level_values('col').isin(list(col_scaler.index)))].mul(col_scaler)

In [7]:
def report(text,df):
    log_absdf = df.copy()
    log_absdf.loc[log_absdf['val']!=0,'val']=(np.log10(
        np.absolute(
            log_absdf.loc[log_absdf['val']!=0,'val'])))
    print(f"{text}:","[",np.int32(np.min(log_absdf)),",",np.int32(np.max(log_absdf)),"]")

report("Original value",matrix)
report("Row scaling val",matrix0)
report("New Matrix",new_matrix)

Original value: [ -8 , 0 ]
Row scaling val: [ -4 , 4 ]
New Matrix: [ -2 , 2 ]


In [8]:
showme(matrix)

col,constobj,"x(san-diego,chicago)","x(san-diego,new-york)","x(san-diego,topeka)","x(seattle,chicago)","x(seattle,new-york)","x(seattle,topeka)"
row,,,,,,,
_obj,1.0,1.8,2.500000e-09,1.400000e-08,1.7,2.5,1.8
demand(chicago),NaN,1.0,NaN,NaN,1.0,NaN,NaN
demand(new-york),NaN,NaN,1.000000e+00,NaN,NaN,1.0,NaN
demand(topeka),NaN,NaN,NaN,1.000000e+00,NaN,NaN,1.0
supply(san-diego),NaN,1.0,1.000000e+00,1.000000e+00,NaN,NaN,NaN
supply(seattle),NaN,NaN,NaN,NaN,1.0,1.0,1.0


In [9]:
showme(matrix0)

col,constobj,"x(san-diego,chicago)","x(san-diego,new-york)","x(san-diego,topeka)","x(seattle,chicago)","x(seattle,new-york)","x(seattle,topeka)"
row,,,,,,,
_obj,12649.110641,22768.399153,0.000032,0.000177,21503.488089,31622.776602,22768.399153
demand(chicago),NaN,1.000000,NaN,NaN,1.000000,NaN,NaN
demand(new-york),NaN,NaN,1.000000,NaN,NaN,1.000000,NaN
demand(topeka),NaN,NaN,NaN,1.000000,NaN,NaN,1.000000
supply(san-diego),NaN,1.000000,1.000000,1.000000,NaN,NaN,NaN
supply(seattle),NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000


In [10]:
showme(new_matrix)

col,constobj,"x(san-diego,chicago)","x(san-diego,new-york)","x(san-diego,topeka)","x(seattle,chicago)","x(seattle,new-york)","x(seattle,topeka)"
row,,,,,,,
_obj,1.0,150.892012,0.005623,0.013307,146.640677,177.827941,150.892012
demand(chicago),NaN,0.006627,NaN,NaN,0.006819,NaN,NaN
demand(new-york),NaN,NaN,177.827941,NaN,NaN,0.005623,NaN
demand(topeka),NaN,NaN,NaN,75.146020,NaN,NaN,0.006627
supply(san-diego),NaN,0.006627,177.827941,75.146020,NaN,NaN,NaN
supply(seattle),NaN,NaN,NaN,NaN,0.006819,0.005623,0.006627


In [12]:
row_scaler

,val
row,
_obj,12649.110641


In [11]:
col_scaler

,val
col,
"x(san-diego,topeka)",75.146020
constobj,0.000079
"x(seattle,chicago)",0.006819
"x(san-diego,chicago)",0.006627
"x(san-diego,new-york)",177.827941
"x(seattle,new-york)",0.005623
"x(seattle,topeka)",0.006627


## Modify below later

np.log10(524288.0)